In [9]:
from methods import *
# Set up the Chrome driver
main_driver= init_driver()
#headless driver

In [10]:
def login_linkedin_sales_navigator(main_driver,user):

    main_driver.get("https://www.linkedin.com/login?session_redirect=%2Fsales&_f=navigator&fromSignIn=true/1000")
    username,password = get_sales_navigator_creds(user)
    time.sleep(5)

    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[1]/input").send_keys(username)
    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[2]/input").send_keys(password)

    time.sleep(3)

    main_driver.find_element(By.XPATH, "/html/body/div/main/div[2]/form/div[3]/button").click()


In [8]:
main_driver.quit()

In [13]:
login_apollo(driver=main_driver)
go_to_lists(driver=main_driver)


KeyboardInterrupt: 

In [14]:
sort_by_name(driver=main_driver)

In [15]:
def extract_data(row,bucket_name,competitor_domain,competitor_name):
    com_linked_in=''
    linked_in=''
    name = row.findAll('td')[0].text.split(' ')[0]
    # error handling for linkedin
    try:
        linked_in = row.findAll('td')[0].select('a[href]')[0].get('href')
        if 'linkedin' not in linked_in:
            linked_in = row.findAll('td')[0].select('a[href]')[1].get('href')
            if 'linkedin' not in linked_in:
                linked_in = row.findAll('td')[0].select('a[href]')[2].get('href')
                if 'linkedin' not in linked_in:
                    linked_in = row.findAll('td')[0].select('a[href]')[3].get('href')
    except:
        pass    
    title = row.findAll('td')[1].text
    company = row.findAll('td')[2].text
    # error handling for company linkedin
    try:
        company_website = row.findAll('td')[2].select('a[href]')[1].get('href')
    except:
        company_website=""

    try:
        com_linked_in = row.findAll('td')[2].select('a[href]')[1].get('href')
        if 'linkedin' not in com_linked_in:
            com_linked_in = row.findAll('td')[2].select('a[href]')[2].get('href')
        else:
            pass
    except:
        pass
    email = row.findAll('td')[4].text
    industry = row.findAll('td')[6].text
    contact_loc = row.findAll('td')[8].text
    company_loc = row.findAll('td')[7].text.replace('N/A',contact_loc)
    
    company_domain = company_website.replace('https://','').replace('http://','').replace('/','')
    
    # print(name,linked_in,title,company,contact_loc,industry,email)
    return {'Name': name,
            'Email':email,
            'linked_in':linked_in,
            'company':company,
            'com_linked_in':com_linked_in,
            'title': title,
            'industry':industry,
            'contact_loc':contact_loc,
            'company_loc':company_loc,
            'company_domain':company_domain,
            }

In [16]:
def get_data(driver,page_count,competitor_domain,competitor_name,industry,country,date):
    industry = industry.lower().replace(' ','-')
    country = country.lower().replace(' ','-')
    date = date.replace('_','-').replace('/','-')
    csv_filename = f"{industry}-{country}-{date}.csv"
    bucket_name = f"{industry}-{country}-{date}"
    data = []
    for i in range(1,page_count+1):
        time.sleep(5)
        the_soup = BeautifulSoup(driver.page_source, 'html.parser')
        table=the_soup.findAll('tbody')
        for row in table:
            data.append(extract_data(row,bucket_name,competitor_domain,competitor_name))
        if i<page_count:
            driver.find_element(By.XPATH,"/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]").click()
    return data,bucket_name,csv_filename

In [17]:
from random import randint


def automate1(driver, url, list_name,num):
    driver.get(url)
    time.sleep(randint(5, 10))

    for i in range(num):

        #click on select all
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[1]/div/button').click()
        time.sleep(randint(5, 10))

        #select all
        driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/div/div/a[1]').click()
        time.sleep(randint(5, 10))

        # #open add to list
        # driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[4]/button[3]').click()
        # time.sleep(randint(5, 10))

        # #click on add to list
        # driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/div/div/a').click()
        # time.sleep(randint(5, 10))

        # click on save button
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div[1]/div[3]/span[1]/button').click()
        time.sleep(randint(5, 10))

        #send list name
        driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[2]/form/div[1]/div[2]/div/div/div/div[2]/div/div/div/div/div/div[1]/input').send_keys(list_name)
        time.sleep(randint(5, 10))

        #click on save button
        driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div/div/div[3]/button[1]').click()
        time.sleep(randint(5, 10))

        #refresh prospects
        driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div[2]/div[2]').click()
        time.sleep(randint(5, 10))


        # # next page
        # driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]').click()
        # time.sleep(randint(5, 10))

In [18]:
def extract_data(row,bucket_name,competitor_domain,competitor_name):
    com_linked_in=''
    linked_in=''
    name = row.findAll('td')[0].text.split(' ')[0]
    # error handling for linkedin
    try:
        linked_in = row.findAll('td')[0].select('a[href]')[0].get('href')
        if 'linkedin' not in linked_in:
            linked_in = row.findAll('td')[0].select('a[href]')[1].get('href')
            if 'linkedin' not in linked_in:
                linked_in = row.findAll('td')[0].select('a[href]')[2].get('href')
                if 'linkedin' not in linked_in:
                    linked_in = row.findAll('td')[0].select('a[href]')[3].get('href')
    except:
        pass    
    title = row.findAll('td')[1].text
    company = row.findAll('td')[2].text
    # error handling for company linkedin
    try:    
        company_website = row.findAll('td')[2].select('a[href]')[1].get('href')
    except:
        company_website=""

    try:
        com_linked_in = row.findAll('td')[2].select('a[href]')[1].get('href')
        if 'linkedin' not in com_linked_in:
            com_linked_in = row.findAll('td')[2].select('a[href]')[2].get('href')
            if 'linkedin' not in com_linked_in:
                com_linked_in = row.findAll('td')[2].select('a[href]')[3].get('href')
                if 'linkedin' not in com_linked_in:
                    com_linked_in = row.findAll('td')[2].select('a[href]')[4].get('href')
    except:
        pass
    email = row.findAll('td')[4].text
    industry = row.findAll('td')[6].text
    contact_loc = row.findAll('td')[8].text
    company_loc = row.findAll('td')[7].text.replace('N/A',contact_loc)
    
    company_domain = company_website.replace('https://','').replace('http://','').replace('/','')
    
    # print(name,linked_in,title,company,contact_loc,industry,email)
    return {'Name': name,
            'Email':email,
            'linked_in':linked_in,
            'company':company,
            'com_linked_in':com_linked_in,
            'title': title,
            'industry':industry,
            'contact_loc':contact_loc,
            'company_loc':company_loc,
            'company_domain':company_domain
            }

In [14]:
link="https://app.apollo.io/#/people?finderViewId=5b6dfc5a73f47568b2e5f11c&personTitles[]=owner&personTitles[]=Founder&personTitles[]=founder%20and%20ceo&personTitles[]=CEO&personTitles[]=co%20founder&personTitles[]=CMO&personTitles[]=managing%20director&personTitles[]=manager&personTitles[]=Dean&personTitles[]=medical%20director&personTitles[]=md%20phd&page=1&personLocations[]=United%20Kingdom&organizationNumEmployeesRanges[]=1%2C10&organizationNumEmployeesRanges[]=11%2C20&organizationNumEmployeesRanges[]=21%2C50&organizationNumEmployeesRanges[]=51%2C100&organizationNumEmployeesRanges[]=101%2C200&organizationNotIndustryTagIds[]=5567e0c97369640d2b3b1600&organizationNotIndustryTagIds[]=5567cd4773696439b10b0000&organizationNotIndustryTagIds[]=5567cd467369644d39040000&contactEmailStatusV2[]=likely_to_engage&contactEmailStatusV2[]=verified&qOrganizationKeywordTags[]=IVF&qOrganizationKeywordTags[]=Fertility&qOrganizationKeywordTags[]=Infertility&qOrganizationKeywordTags[]=Surrogacy&qOrganizationKeywordTags[]=Test%20Tube%20baby&qOrganizationKeywordTags[]=Gynecology&qOrganizationKeywordTags[]=In%20vitro%20fertilization%20(IVF)&qOrganizationKeywordTags[]=in%20vitro%20fertilization&qOrganizationKeywordTags[]=obstetrics&qOrganizationKeywordTags[]=Reproductive&qOrganizationKeywordTags[]=Fertility%20Clinic&qOrganizationKeywordTags[]=Male%20Fertility&qOrganizationKeywordTags[]=Ovulation%20Test&qOrganizationKeywordTags[]=Infertility%20Treatment&includedOrganizationKeywordFields[]=tags&includedOrganizationKeywordFields[]=name&finderTableLayoutId=65c9cf81d945150300252849&sortByField=person_name.raw&sortAscending=true&prospectedByCurrentTeam[]=no"
list_name="IVF UK 1"
automate1(main_driver,link, list_name,24)

KeyboardInterrupt: 

In [19]:
# pages=get_page_count(driver=main_driver)
pages=21
competitor_name=""
competitor_domain=""
industry="cannabis-3"
country="us"
date="17-04-2024"


In [20]:
main_data,bucket_name,csv_file_name=get_data(driver=main_driver,page_count=pages,competitor_domain=competitor_domain,competitor_name=competitor_name,industry=industry,country=country,date=date)

In [5]:
create_s3_bucket(s3=init_s3(),bucket_name="real-estate-vermont-us-29-02-24")

True

In [21]:
csv_url=write_to_csv(data=main_data,filename=csv_file_name,bucket_name=bucket_name)

In [22]:
print(csv_url)

https://cannabis-3-us-17-04-2024.s3.amazonaws.com/cannabis-3-us-17-04-2024.csv


In [7]:
login_semrush(main_driver)

In [154]:
screenshot_to_s3(s3=init_s3(),driver=main_driver,bucket_name=bucket_name,domain=competitor_domain,region=country)

True

In [39]:
s3=init_s3()

In [14]:
update_screenshots_to_s3(s3=init_s3(),driver=main_driver,bucket_name=bucket_name,data=main_data,region=country)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010480a814 chromedriver + 4040724
1   chromedriver                        0x0000000104802a18 chromedriver + 4008472
2   chromedriver                        0x00000001044756a8 chromedriver + 284328
3   chromedriver                        0x00000001044acc14 chromedriver + 510996
4   chromedriver                        0x00000001044d71c0 chromedriver + 684480
5   chromedriver                        0x00000001044d2720 chromedriver + 665376
6   chromedriver                        0x00000001044d209c chromedriver + 663708
7   chromedriver                        0x0000000104446d74 chromedriver + 93556
8   chromedriver                        0x00000001047cfa3c chromedriver + 3799612
9   chromedriver                        0x00000001047d3f1c chromedriver + 3817244
10  chromedriver                        0x00000001047b8298 chromedriver + 3703448
11  chromedriver                      

In [185]:


main_driver.quit()

In [8]:
screenshot_to_s3(s3=init_s3(),driver=main_driver,bucket_name="real-estate-ny-us-19-02-2024",domain="www.peterashe.com",region="us")

True

In [133]:
data = update_competitor_details(data=main_data[1000:],competitor_name="Eagle Rock Properties",competitor_domain="www.eaglerockproperties.com",bucket_name="real-estate-ny-us-20-02-2024")


In [146]:
data=update_competitor_and_company_details(data=main_data[1000:],competitor_name="Eagle Rock Properties",competitor_domain="www.eaglerockproperties.com",bucket_name="real-estate-ny-us-20-02-2024")

In [151]:
write_to_csv(data=data,filename=csv_file_name,bucket_name=bucket_name)

'https://real-estate-ny-us-20-02-2024.s3.amazonaws.com/real-estate-ny-us-20-02-2024.csv'

In [1]:
# read csv into dictionary
from pandas import read_csv
df = read_csv('real-estate-ny-us-19-02-2024.csv')
data = df.to_dict(orient='records')

In [30]:
lookup_data=read_csv("clicked.csv").to_dict(orient='records')


In [25]:
email=str(data[4]["Email"]).lower().strip()
email1=str(lookup_data[0]["Email"]).lower().strip()

email==email1

True

In [31]:
linkedin_url=[]
for item in lookup_data:
    email=str(item["Email"]).lower().strip()
    for data_item in data:
        email1=str(data_item["Email"]).lower().strip()
        if email==email1:
            print(email)
            print(data_item["linked_in"])
            print("found")
            linkedin_url.append(data_item["linked_in"])
            break
    else:
        print("not found")
        print(email)
        print(email1)
        print("not found")
        break

akleiman@nexusci.com
http://www.linkedin.com/in/aaron-kleiman-bb2502122
found
abickoff@sterlingprojectdevelopment.com
http://www.linkedin.com/in/adam-bickoff-4309548
found
a.albert@tmdre.com
http://www.linkedin.com/in/adrienne-albert-59798413
found
alec@olcny.com
http://www.linkedin.com/in/alec-ornstein-2530666
found
asvetlakou@ripcony.com
http://www.linkedin.com/in/alex-svetlakou-a90a86161
found
acaiola@bandlmanagement.com
http://www.linkedin.com/in/alfred-caiola-35112419
found
alyssa.thomas@rsequity.com
http://www.linkedin.com/in/alyssa-thomas-43b69723
found
awetzler@sterlingprojectdevelopment.com
http://www.linkedin.com/in/andrewwetzler
found
carol@velocitytitle.com
#/contacts/65d2eac4db6b25000166a096
found
carol@premierequities.com
http://www.linkedin.com/in/carol-hensley-chester-8749bb7
found
charlotte.banghart@rsequity.com
http://www.linkedin.com/in/charlotte-banghart-9b18027b
found
chris.murray@redstoneequity.com
http://www.linkedin.com/in/chris-murray-9642a09
found
christina@ve

In [34]:
main_driver.get("https://www.linkedin.com/in/")

In [84]:



for i in range(31,40):
    main_driver.get(linkedin_url[i])
    time.sleep(10)

KeyboardInterrupt: 

In [80]:
create_s3_bucket(s3=init_s3(),bucket_name="test-bucket-versioning-hotbit")

AttributeError: 'S3' object has no attribute 'BucketVersioning'

In [ ]:
import imaplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Function to connect to Gmail
def connect_to_gmail(username, password):
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(username, password)
    return mail

# Function to search for email thread
def search_email(mail, search_query):
    mail.select('inbox')
    result, data = mail.search(None, '(SUBJECT "{}")'.format(search_query))
    return data[0].split()[-1]  # Assuming the latest email in the thread is what we want

# Function to fetch email
def fetch_email(mail, email_id):
    result, data = mail.fetch(email_id, '(RFC822)')
    raw_email = data[0][1]
    return email.message_from_bytes(raw_email)

# Function to compose email
def compose_email(to, subject, body, references=None):
    msg = MIMEMultipart()
    msg['From'] = "your_email@gmail.com"
    msg['To'] = to
    msg['Subject'] = subject
    
    # If there are references, add them to the email headers
    if references:
        msg['References'] = references
        msg['In-Reply-To'] = references

    # Create a MIMEText object with HTML content
    print(type(body))
    print(body)
    if isinstance(body, bytes):
        body = body.decode('utf-8')
    html_body = MIMEText(body, 'html')
    msg.attach(html_body)
    
    return msg

# Function to save email as draft
def save_as_draft(mail, draft):
    mail.append('INBOX', None, None, draft)

# Function to draft initial email
def draft_initial_email(mail, to, subject, body):
    draft = compose_email(to, subject, body)
    save_as_draft(mail, draft)

# Function to draft trailing email
def draft_trailing_email(mail, to, subject, body, references):
    draft = compose_email(to, subject, body, references)
    save_as_draft(mail, draft)


def draft_email_imap(data, initial, follow_up):
    # Example usage
    username = 'avora@hotbitdigital.com'
    password = 'unsw vvjr nzbt yhwd'

    mail = connect_to_gmail(username, password)
    if initial:
    # Draft initial email
        to = data['receiver']
        initial_subject = f"Discover How {data['company']} Can Soar in Vermont!"
        initial_body = '<html><body><p>Dear recipient,</p><p>This is the initial email...</p></body></html>'
        draft_initial_email(mail, to, initial_subject, initial_body)

    if follow_up:
        # Draft trailing emails with different subject lines in the same thread
        initial_subject = f"Discover How {data['company']} Can Soar in Vermont!"
        initial_references = search_email(mail, initial_subject)
        trailing_subjects = [f"Quick Check-In: Enhancing {data['company']}'s Visibility", 'Follow-up 2 Subject', 'Follow-up 3 Subject', 'Follow-up 4 Subject', 'Follow-up 5 Subject']


        draft_trailing_email(mail, data['receiver'], trailing_subjects[follow_up-1], get_trailing_body(data,follow_up=follow_up), initial_references)
